In [ ]:
# Import libraries 

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np
from PIL import Image, ImageDraw
import tkinter as tk

In [ ]:
# Define a CNN model in PyTorch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(12*12*64, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(-1, 12*12*64)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

In [ ]:
# Load MNIST data and apply transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transform),
    batch_size=128, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transform),
    batch_size=128, shuffle=False)

In [ ]:
# Create the model, optimizer, and loss function
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Train the model
for epoch in range(1, 11):
    train(epoch)

# Save the model
torch.save(model.state_dict(), "mnist_model.pth")

# Load the model
model = Net()
model.load_state_dict(torch.load("mnist_model.pth"))
model.eval()

# Digit Recognition: Draw and Predict

Just remove the comments (''')  and run it

In [ ]:
'''
def predict_digit(img):
    # Resize and invert colors
    img = img.resize((28, 28))
    img = img.convert('L')
    img = np.array(img)
    img = img.reshape(1, 1, 28, 28)
    img = img / 255.0

    # Convert to PyTorch tensor
    img = torch.FloatTensor(img)

    # Predicting the digit
    with torch.no_grad():
        output = model(img)
    _, predicted = torch.max(output, 1)
    return predicted.item()

def draw(event):
    x = event.x
    y = event.y
    draw_canvas.line([(x, y), (x+1, y+1)], fill='black', width=8)

def clear():
    global image, draw_canvas
    image = Image.new("RGB", (200, 200), (255, 255, 255))
    draw_canvas = ImageDraw.Draw(image)
    canvas.delete("all")

def predict():
    digit = predict_digit(image)
    label.configure(text=str(digit))



root = tk.Tk()

canvas = tk.Canvas(root, width=200, height=200, bg='white')
canvas.grid(row=0, column=0, pady=2, sticky=tk.W)

button_clear = tk.Button(root, text="Clear", command=clear)
button_clear.grid(row=1, column=0, pady=2)

button_predict = tk.Button(root, text="Predict", command=predict)
button_predict.grid(row=1, column=1, pady=2)

label = tk.Label(root, text="", font=("Helvetica", 48))
label.grid(row=0, column=1, pady=2, padx=2)

canvas.bind("<B1-Motion>", draw)

image = Image.new("RGB", (200, 200), (255, 255, 255))
draw_canvas = ImageDraw.Draw(image)

root.mainloop()

'''